<a href="https://colab.research.google.com/github/sqpoonz/esomatec/blob/main/Modelmaker_esomatec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 621 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 30.2 MB/s 
     |████████████████████████████████| 840 kB 19.7 MB/s 
     |████████████████████████████████| 87 kB 2.9 MB/s 
     |████████████████████████████████| 3.4 MB 34.3 MB/s 
     |████████████████████████████████| 1.2 MB 39.1 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 6.3 MB 27.4 MB/s 
     |████████████████████████████████| 213 kB 44.1 MB/s 
     |████████████████████████████████| 1.1 MB 37.4 MB/s 
     |████████████████████████████████| 120 kB 54.9 MB/s 
     |████████████████████████████████| 25.3 MB 88 kB/s 
     |████████████████████████████████| 352 kB 49.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.8 MB/s 
     |████████████████████████████████| 47.6 MB 37 kB/s 
     |████████████████████████████████| 99 kB 7.0 MB/s 
     |████████████████████████████████| 208 kB 53.2 MB/s 


In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
!mkdir /content/images
!unzip -q images.zip -d /content/images/all
!mkdir /content/images/train; mkdir /content/images/validation

In [ ]:
!wget https://raw.githubusercontent.com/sqpoonz/esomatec/main/split.py
!python split.py

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'images/train',
    'images/train',
    ['1', '9']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'images/validation',
    'images/validation',
    ['1', '9']
)

In [ ]:
spec = model_spec.get('efficientdet_lite0')

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

In [ ]:
model.evaluate(val_data)

In [ ]:
model.export(export_dir='.', tflite_filename='custom.tflite')

In [ ]:
model.evaluate_tflite('custom.tflite', val_data)

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('custom.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Compile the model for EdgeTPU

Finally, we'll compile the model using `edgetpu_compiler` so that the model can run on [Google Coral EdgeTPU](https://coral.ai/).

We start with installing the EdgeTPU compiler on Colab.

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  59000      0 --:--:-- --:--:-- --:--:-- 59000
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,722 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,398 kB]
Ign:9 https://developer.download.nvidia.com/comput

In [ ]:
NUMBER_OF_TPUS = 5

!edgetpu_compiler custom.tflite --num_segments=$NUMBER_OF_TPUS

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 3127 ms.

Input model: android.tflite
Input size: 4.22MiB
Output model: android_edgetpu.tflite
Output size: 5.57MiB
On-chip memory used for caching model parameters: 4.21MiB
On-chip memory remaining for caching model parameters: 3.29MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 267
Operation log: android_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 264
Number of operations that will run on CPU: 3
See the operation log file for individual operation details.
Compilation child proces

In [ ]:
populator_dst = metadata.MetadataPopulator.with_model_file('custom_edgetpu.tflite')

with open('android.tflite', 'rb') as f:
  populator_dst.load_metadata_and_associated_files(f.read())

populator_dst.populate()
updated_model_buf = populator_dst.get_model_buffer()

In [ ]:
# Download the TFLite model compiled for EdgeTPU to your local computer.
from google.colab import files
files.download('custom_edgetpu.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>